# AC290 Extreme Computing: Project-based High Performance Distributed and Parallel Systems

## Module 1: Influence Maximization Problem

<img src='img/AC290_IMP_graph1.png' style="width: 500px;">
<p style="text-align: center;">Yelp Reviewer Network</p>

In [25]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt

## 1. Python

### 1.1. Time Profiling

We'll examine two different ways to profile run time of a function by using the following function below:

An example from http://www.huyng.com/posts/python-performance-analysis/

In [14]:
def primes(n): 
    if n == 2:
        return [2]
    elif n < 2:
        return []
    s = range(3,n+1,2)
    mroot = n ** 0.5
    half = (n + 1) / 2 - 1
    i = 0; m = 3;
    while m <= mroot:
        if s[i]:
            j = (m * m-3) / 2
            s[j] = 0
            while j < half:
                s[j] = 0
                j += m
        i = i + 1
        m = 2 * i + 3
    return [2]+[x for x in s if x]

The simplest method to evaluate your function's run time is to use Python's time module and manually print out the run time of your function.  This is a quick and dirty way to measure your function's performance.

In [7]:
import time

In [23]:
start = time.time()
prime_nums = primes(int(10e6))
duration = time.time() - start
print "%f Seconds" % duration

2.607845 Seconds


Using this approach, we can store the run time of the your function for analysis later on (mean, standard deviation, etc.). 

Expanding on this simple method, let's create a function decorator that will automate this process every time we want to evaluate a function's time performance.

In [8]:
from functools import wraps

def timefn(func):
    @wraps(func)
    def calc_time(*args, **kwargs):
        t1=time.time()
        result = func(*args,**kwargs)
        t2=time.time()
        print "@timefn: %.5f Seconds" % (t2-t1)
        return result
    return calc_time

In [15]:
@timefn
def primes(n): 
    if n == 2:
        return [2]
    elif n < 2:
        return []
    s = range(3,n+1,2)
    mroot = n ** 0.5
    half = (n + 1) / 2 - 1
    i = 0; m = 3;
    while m <= mroot:
        if s[i]:
            j = (m * m-3) / 2
            s[j] = 0
            while j < half:
                s[j] = 0
                j += m
        i = i + 1
        m = 2 * i + 3
    return [2]+[x for x in s if x]

In [13]:
prime_nums = primes(int(10e6))

@timefn: 2.71378 Seconds


Another simple method is to use iPython's %time magic function.

In [26]:
%time prime_nums = primes(int(10e6))

CPU times: user 2.9 s, sys: 53 ms, total: 2.95 s
Wall time: 2.96 s


### 1.2. Set vs List vs Tuple

>"One of the most important things in writing efficient programs is understanding the guarantees of the data structures you use.  In fact, a large part of performant programming is understanding what questions you are trying to ask of your data and picking a data structure that you can answer these questions quickly." - Gorelick & Ozsvald

Python provides 4 basic built-in data types - list, set, tuple, and dictionary (set without values).  We will compare the run time performance of each data types for different set of problems in order to understand which data structure provides the best solution to each unique problem.

In [9]:
n = int(10e6)

In [10]:
python_list = list(xrange(n))
python_set = set(xrange(n))
python_tuple = tuple(xrange(n))

### 1.2.1. Membership

List - dynamic array

In [11]:
start = time.time()
print n in python_list
duration = time.time() - start
print "%f Seconds" % duration

False
0.794647 Seconds


Tuple - static array

In [12]:
start = time.time()
print n in python_tuple
duration = time.time() - start
print "%f Seconds" % duration

False
0.666547 Seconds


Tuples are cached by Python; therefore, performance may significantly improve after the first time a tuple is called.

In [13]:
# Second Run
start = time.time()
print n in python_tuple
duration = time.time() - start
print "%f Seconds" % duration

False
0.234677 Seconds


Set.  

A set (or dictionary) is ideal data structure if your data does not have intrinsic order and have unique elements.

In [14]:
start = time.time()
print n in python_set
duration = time.time() - start
print "%f Seconds" % duration

False
0.000159 Seconds


### 1.2.2. Iteration

List

In [15]:
start = time.time()
for i in python_list:
    i
duration = time.time() - start
print "%f Seconds" % duration

0.850017 Seconds


Tuple

In [18]:
start = time.time()
for i in python_tuple:
    i
duration = time.time() - start
print "%f Seconds" % duration

0.807556 Seconds


Set

In [19]:
start = time.time()
for i in python_set:
    i
duration = time.time() - start
print "%f Seconds" % duration

1.397327 Seconds


### 1.3. List Comprehension vs For Loop

For Loop

In [33]:
start = time.time()
list_forloop = list()
for i in xrange(n):
    list_forloop.append(i)
duration = time.time() - start
print "%f Seconds" % duration

2.269928 Seconds


List Comprehension

In [34]:
start = time.time()
list_comprehension = [i for i in xrange(n)]
duration = time.time() - start
print "%f Seconds" % duration

1.578796 Seconds


### 1.4. Mutability

### 1.4.1. Comparing List vs Tuple

In [35]:
my_list = ['a','b','c','d']
my_tuple = ('a','b','c','d')

In [36]:
my_list[0] = 'A'
my_list

['A', 'b', 'c', 'd']

Let's attempt to change the first element of my_tuple.

In [37]:
my_tuple[0] = 'A'
my_tuple

TypeError: 'tuple' object does not support item assignment

### 1.4.2. Copying Lists

In [38]:
my_list = ['a','b','c','d']

In [39]:
my_list2 = my_list

Let's change the second element of my_list2

In [40]:
my_list2[1] = 'B'
my_list2

['a', 'B', 'c', 'd']

We ended up changing the second element in my_list to 'B'.

In [41]:
my_list

['a', 'B', 'c', 'd']

To avoid this problem, there are three ways to copy a list that will leave the original list unperturbed. 

###### Make element-wise copy to a new list

In [42]:
my_list = ['a','b','c','d']
my_list2 = my_list[:]
my_list2[1] = 'B'
print "my_list:", my_list, "| my_list2:", my_list2

my_list: ['a', 'b', 'c', 'd'] | my_list2: ['a', 'B', 'c', 'd']


##### Create a new list out of the original list

In [44]:
my_list = ['a','b','c','d']
my_list2 = list(my_list)
my_list2[1] = 'B'
print "my_list:", my_list, "| my_list2:", my_list2

my_list: ['a', 'b', 'c', 'd'] | my_list2: ['a', 'B', 'c', 'd']


###### Import copy module and make a deepcopy

In [45]:
my_list = ['a','b','c','d']
import copy
my_list2 = copy.deepcopy(my_list)
my_list2[1] = 'B'
print "my_list:", my_list, "| my_list2:", my_list2

my_list: ['a', 'b', 'c', 'd'] | my_list2: ['a', 'B', 'c', 'd']


### 1.4. Line Profiling

In [6]:
import line_profiler
import IPython
ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)

Let's use the primes function that we used earlier to demonstrate what line-profiler can do for us.

In [7]:
def primes(n): 
    if n==2:
        return [2]
    elif n<2:
        return []
    s=range(3,n+1,2)
    mroot = n ** 0.5
    half=(n+1)/2-1
    i=0
    m=3
    while m <= mroot:
        if s[i]:
            j=(m*m-3)/2
            s[j]=0
            while j<half:
                s[j]=0
                j+=m
        i=i+1
        m=2*i+3
    return [2]+[x for x in s if x]

In [8]:
%lprun -f primes primes(100)

For detailed discussions about optimizing performance in Python, checkout [High Performance Python](http://shop.oreilly.com/product/0636920028963.do)
<img src="img/lrg.jpg" style="max-width: 200px;">

## 2. NetworkX

NetworkX is a popular Python library used to model networks including social network graphs.  Using Networkx, we can model both undirected and directed graphs.

For the Influence Maximization Problem, we will provide you with a pre-built graph of Yelp Reviewer network and ask you to use Networkx to implement the Independent Cascade function. 

### 2.1. Installation

If you installed Python through Anaconda distribution, you should already have NetworkX in your Python environment.  

If you do not have it, you may run the following command below.

In [28]:
import networkx as nx

In [5]:
nx.__version__

'1.9.1'

You should use NetworkX version 1.8 or above.

### 2.2. Examples

Creating a sample graph

In [83]:
random_graph = nx.erdos_renyi_graph(n=20,p=0.6,seed=42)

Finding nodes

In [84]:
random_graph.nodes()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [85]:
random_graph[12]

{0: {},
 2: {},
 3: {},
 4: {},
 6: {},
 7: {},
 10: {},
 11: {},
 13: {},
 16: {},
 17: {},
 19: {}}

Finding edges

In [86]:
random_graph.edges()

[(0, 2),
 (0, 3),
 (0, 4),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 16),
 (0, 17),
 (0, 18),
 (1, 2),
 (1, 5),
 (1, 6),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 15),
 (1, 17),
 (1, 18),
 (2, 5),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 15),
 (2, 16),
 (2, 17),
 (2, 18),
 (3, 6),
 (3, 8),
 (3, 9),
 (3, 12),
 (3, 16),
 (3, 17),
 (3, 18),
 (3, 19),
 (4, 5),
 (4, 8),
 (4, 10),
 (4, 12),
 (4, 13),
 (4, 14),
 (4, 15),
 (4, 16),
 (4, 17),
 (4, 19),
 (5, 6),
 (5, 8),
 (5, 9),
 (5, 10),
 (5, 11),
 (5, 14),
 (5, 15),
 (5, 16),
 (5, 18),
 (6, 8),
 (6, 11),
 (6, 12),
 (6, 14),
 (6, 15),
 (6, 16),
 (6, 19),
 (7, 8),
 (7, 9),
 (7, 12),
 (7, 15),
 (7, 17),
 (7, 19),
 (8, 9),
 (8, 10),
 (8, 11),
 (8, 15),
 (8, 16),
 (8, 19),
 (9, 10),
 (9, 11),
 (9, 14),
 (9, 15),
 (9, 16),
 (9, 17),
 (9, 19),
 (10, 11),
 (10, 12),
 (10, 14),
 (10, 15),
 (10, 18),
 (10, 19),
 (11, 12),
 (11, 13),
 (11, 14),
 (11, 15),
 (11, 16),
 (11, 17),
 (11, 18),
 (12, 13),
 (

In [88]:
random_graph.edge[0][3]

{}

Storing edge weights

In [90]:
for s,t in random_graph.edges_iter():
    random_graph[s][t]['weight'] = np.random.uniform()

In [91]:
random_graph.edge[0][3]

{'weight': 0.5719586317584954}

For more examples, please explore the [NetworkX documentation](http://networkx.readthedocs.org/en/stable/index.html).

### 2.3. Exploring Yelp Review Network 

Before we scale up the Influence Maximization Problem on larger network, we will work with a "toy" network that is a sample of North Carolina reviewer network.  This network contains 240 nodes and 920 edges.

By week 3, you will solve the IMP on this smaller network, and by week 4, you will implement your algorithms on the larger network, which contains over 350 thousand nodes and 4 million edges.

In [16]:
import json
from networkx.readwrite import json_graph

In [17]:
with open("graph/nc_mini.json", "r") as graph_data:
    graph_data = json.load(graph_data)
    NC_digraph = json_graph.node_link_graph(graph_data)

In [18]:
NC_digraph.nodes()

[u'PpkKVodWC0sdn74TbHQLzA',
 u'9kNIqxBAFKkPVIN_7MRtOg',
 u'lhIEvJs8o09n6-zdffln0g',
 u'GrA3BLjRjAKvFYwSqN_4cg',
 u'_-S38fYbXdf2J9n7se30jg',
 u'VEtVKHZIEjq_YTd8ZPBu-g',
 u'UIbW6xWBczva9e4fcuV8JA',
 u'I_krsLQejP6-Ta_6xiigNg',
 u'emoQY8neOSJm-xwqh4xVfQ',
 u'yfQNTgMn8UfMM0Ibr0OwoQ',
 u'AUDvPJ5LxB5m_oMiFl6Xww',
 u'FB_kqMLsd8BKIydQUUC_Ag',
 u'ltpnxJxM446ZEDrrzQ5anA',
 u'GRiX5F5zgJiQQ1NyV3B6Cg',
 u'OBS3D3WRVNT9tHhdyS0D6Q',
 u'Wvtn9aKFXGcIeKaakFsYLA',
 u'kcAGTivq8YNmCXe2RBAyIw',
 u'fW0eVXKzwg8Bgbl2AJDEjg',
 u'kxRnhpUgooticN9DimR-CQ',
 u'Nnr__fWrInIRKv8WLCh9nA',
 u'glzRad3_l42hALvjB9p97w',
 u'Rm3ZSxxp_ErscqmBoa8bdg',
 u'5hhvqWu2vCo4HSKFw4H0pQ',
 u'4itIWbWKDMXef7VzAQ41-g',
 u'I9-qaI_sfTJzeG52WZMdkQ',
 u'bYm-qoFlgzSwbKHH0StXMg',
 u'n9OkjFjUBIQ5zze7nLuQog',
 u'y57yAzmqx5cR0cfeFE91Sg',
 u'RI9E2EPqMaO3PRZ_llj6tg',
 u'5uWopFOiNzO1041yb4XRPQ',
 u'HFl6MLArBmfl35WYwtS0gQ',
 u'uL7TIJ56Eb9Rx4nkt2ej_Q',
 u'VVTdlxNhx_O4VBJyu2OInQ',
 u'M9H6p4hm2utu1G0ipV80Ug',
 u'ilMy7QjAbz6Uzdqw0lluBw',
 u'ttpwWJ9fOohzKEyxL

In [19]:
print NC_digraph.number_of_nodes(), "nodes"

240 nodes


In [20]:
print NC_digraph.number_of_edges(), "edges"

1061 edges


In [21]:
NC_digraph.succ['Z1FWaNNO8oxaHLYB9XhQDg']

{u'C1BIq8rhj0ZG2t3tV6pKFg': {u'weight': 1},
 u'UCK9qP7EyBYWHZiN0ckdpw': {u'weight': 1},
 u'e2j5nTQ2dnkB--pJakcUSA': {u'weight': 1},
 u'gD2IlYNxzvFOuRMGqH96lQ': {u'weight': 1},
 u'ltpnxJxM446ZEDrrzQ5anA': {u'weight': 1},
 u'xqqAzP6zjtJbyegKR1Md9A': {u'weight': 1}}

In [22]:
NC_digraph.edges(data=True)

[(u'PpkKVodWC0sdn74TbHQLzA', u'S3HZF5aANmhZoMkFkPMdqQ', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'Y--yRCncNlQCNULP8I5h6Q', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'9P_5UGjc3t3Bo_yJ5A7iAQ', {u'weight': 2}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'FB_kqMLsd8BKIydQUUC_Ag', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'3GteArKIVRlJZjjRqqwgaQ', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'NzWLMPvbEval0OVg_YDn4g', {u'weight': 3}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'-_1ctLaz3jhPYc12hKXsEQ', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'5hhvqWu2vCo4HSKFw4H0pQ', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'UmCAhWcU_abaaESenKUiLw', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'tm_HvdZmUMGht1-xjGQF7Q', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'0SuIvO9hz82-tuKEe2z35A', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'd8y98FiNtI8OQFAp2tvoKg', {u'weight': 1}),
 (u'PpkKVodWC0sdn74TbHQLzA', u'ts7EG6Zv2zdMDg29nyqGfA', {u'weight': 2}),
 (u'PpkKVodWC0sdn74TbHQLzA', u's9I4KMcrHKki44g_eTrY

### 2.2. Visualizing the Graph

In [26]:
def print_graph(Graph, S1=None):
    plt.figure(figsize=(16,10))
    color_map = {1: 'b', 0: 'r'}
    pos = nx.random_layout(Graph)
    
    if S1:
        nx.draw_networkx(Graph, pos, with_labels=False, node_size=100, node_shape='.',
                linewidth=None, width=0.2, edge_color='y', 
                node_color=[color_map[Graph.node[node]['action']] for node in Graph],
                edgelist=reduce(lambda x,y: x+y,[Graph.edges(node) for node in S1]))
        nx.draw_networkx_nodes(Graph, pos, nodelist=S1, node_color="b", node_size=150, 
                              node_shape="*", label="Initial Set")
        plt.legend()
    else:
        nx.draw_networkx(Graph, pos, with_labels=False, node_size=100, node_shape='.',
                linewidth=None, width=0.2, edge_color='y', 
                 node_color=[color_map[Graph.node[node]['action']] for node in Graph])
        
    plt.xlim(-0.05,1.05)
    plt.ylim(-0.05,1.05)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [27]:
print_graph(NC_digraph)

NameError: global name 'nx' is not defined